In [1]:
import numpy as np
from simple.hurst import HurstRS, HurstDMA, HurstDSOD
from simple.funcs import hurst
from simple.pretty import prun
from simple.numpy import rolling_apply
from simple.plotly import chartFigure, updateLines

In [2]:
period = 1024

In [3]:
A = (np.random.rand(period*2)-0.4).cumsum()
B = np.random.rand(period*4) + A[-1]
T = np.hstack((A, B))
len(T)

6144

In [4]:
fig = chartFigure(Price=T, rows=2, HurstDSOD={'row': 2}, hurstSeries={'row': 2}, HurstRS={'row': 2}, HurstDMA={'row': 2})
fig

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> Price <i style="color:#fc99…

In [5]:
def hurstSeries(X: np.array, Period: int) -> np.array:
    x = rolling_apply(lambda x: hurst(x), Period, X, n_jobs=128, progress=True)
    x[:period] = x[period]
    return x

In [6]:
h = hurstSeries(T, period)
updateLines(fig, hurstSeries=h)

  0%|          | 0/5120 [00:00<?, ?it/s]

In [7]:
h = HurstDSOD(T, period)
updateLines(fig, HurstDSOD=h)

In [8]:
h = prun(lambda src, period: HurstRS(np.diff(src, prepend=0), period), T, period)
updateLines(fig, HurstRS=h)

  0%|          | 0/256 [00:00<?, ?it/s]

In [9]:
h = prun(HurstDMA, T, period)
updateLines(fig, HurstDMA=h)

  0%|          | 0/256 [00:00<?, ?it/s]